# Example 3 - Native data acquisition
In this tutorial we run a measurement with a custom data aquisition loop, and we push the data directly to the QHarbor platform.

You can download this jupyter notebook and more examples from [here](https://github.com/qEncoder/dataQruiser/tree/main/examples).

## Define mock instruements

In [1]:
# Here we make the mock instruments with qcodes for convenience
from qdrive.measurement.data_collector import data_collector, from_QCoDeS_parameter
from qdrive.dataset.dataset import dataset
from qdrive.testing.qcodes_test_param import MyCounter

from qcodes.instrument_drivers.mock_instruments import DummyInstrument
from qcodes.parameters import MultiParameter
import time
import numpy as np


try:
    dac = DummyInstrument('dac', gates=['ch1', 'ch2'])
    param = MyCounter("counter")
except: pass

class IQArray(MultiParameter):
    def __init__(self, n_pts):
        # names, labels, and units are the same
        self.n_pts = n_pts
        setpoints_values = tuple(np.linspace(0, n_pts-1, n_pts))
        self.rng = np.random.default_rng(seed=42)
        super().__init__('iq_array', names=('I', 'Q'), shapes=((n_pts,), (n_pts,)),
                         labels=('In phase amplitude', 'Quadrature amplitude'),
                         units=('V', 'V'),
                         # note that EACH item needs a sequence of setpoint arrays
                         # so a 1D item has its setpoints wrapped in a length-1 tuple
                         setpoints=((setpoints_values, ), (setpoints_values,), ),
                         setpoint_names=((("repetition"),), (("repetition"),)),
                         setpoint_labels=((("repetition"),), (("repetition"),)),
                         setpoint_units=((("#"),), (("#"),)),
                         docstring='param that returns two single values, I and Q')

    def get_raw(self):
        return (self.rng.random([self.n_pts]), self.rng.random([self.n_pts]))


/opt/miniconda3/lib/python3.12/site-packages/qdrive/testing/qcodes_test_param.py:10: QCoDeSDeprecationWarning: The `qcodes.tests` module is deprecated, because tests are no longer shipped with QCoDeS. Use "`qcodes.instrument_drivers.mock_instruments` and `qcodes.extensions.DriverTestCase`" as an alternative.
  from qcodes.tests.instrument_mocks import DummyInstrument


## Run a custom measurement loop
This measurement loop pushes data directly to a qdrive dataset. You can see the live dataset populating in the dataQruiser app

In [2]:
from qdrive import Measurement
from qcodes import Station


In [9]:
# Example of a 1D sweep
measurement = Measurement("Native data 1D", scope_name=None)
measurement.register_get_parameter(param, dac.ch1) # parameter, setpoint1, setpoint2, ...
measurement.add_snapshot(Station().snapshot())

with measurement as meas:
    voltages = np.linspace(0,200, 100)
    rng = np.random.default_rng(seed=42)
    for i in voltages:
        dac.ch1.set(i)
        meas.add_result({param : rng.random() + 1, dac.ch1 : dac.ch1.get()})
        time.sleep(0.1)

print(measurement.ds)


Starting measurement with uuid : 684fa6b4-cc2b-4ee1-a21c-21a69d990be4 - Native data 1D
Contents of dataset :: Native data 1D

uuid :: 684fa6b4-cc2b-4ee1-a21c-21a69d990be4
Scope :: z_scope
Ranking :: 0
Files :: 
name         type                  selected version number (version_id)      Maximal version number
-----------  --------------------  --------------------------------------  ------------------------
measurement  FileType.HDF5_NETCDF  0 (1727786375446)                                              0
snapshot     FileType.JSON         0 (1727786375491)                                              0


In [10]:
# Example of a 2D sweep
measurement = Measurement("Native data 2D", scope_name=None)
measurement.register_get_parameter(param, dac.ch1, dac.ch2) # parameter, setpoint1, setpoint2, ...
measurement.add_snapshot(Station().snapshot())

with measurement as meas:
    ch1 = np.linspace(50,200, 20)
    ch2 = np.linspace(10,150, 20)

    for i in ch1:
        dac.ch1.set(i)
        for j in ch2:
            dac.ch2.set(j)
            meas.add_result({param : param.get(), dac.ch1 : dac.ch1.get(), dac.ch2 : dac.ch2.get()})
            time.sleep(0.03)


Starting measurement with uuid : ea41655e-b2f3-49ea-9ecb-c94d840dcb0c - Native data 2D
